### Import Dependencies

In [1]:
import os
import torch
import random
import math
from torch import nn
from torch.nn.modules import activation
import torch.nn.functional as F
import torchvision
import shutil
import numpy as np
from PIL import Image
import time
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix
import math
import TAR_transformer

C:\Users\GillA\.conda\envs\Aseem\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Get Dataloaders

In [2]:
# Empty the gpu cache and check if the gpu is available for use
torch.cuda.empty_cache()
torch.cuda.is_available()

True

In [3]:
### FUNCTION FOR SETTING UP DATA LOADERS
def prepare_dataset(data_dir, batch_size =  64, num_workers = 0, flag = None):
    '''
    Prepares dataloaders for training. Datasets are prepare with a three-cross 
    fold validation approach. 
    
    data_dir - Path to directory containing samples from one rat's training set
               this directory will contain sub-directories which are folders of 
               each class (Dorisflexion, plantarflexion, and pricking of the heel)
    batch_size - how many signals are loaded per batch in the data loaders
    num_workers - number of processes loading batches in parallel
    
    Types
    data_dir - string
    batch_size - int
    num_workers - int
    '''
    # The Test and Validation sets are taken from a single fold
    if flag == "Test" or flag == "Val":
        # Specify where the folder containing the validation or testing signals are for the dataset
        data = torchvision.datasets.DatasetFolder(data_dir, loader = torch.load, extensions = ".pt")

        # Prepare data loaders
        # Signals are loaded in batch sizes as specified 
        loader = torch.utils.data.DataLoader(data, batch_size=batch_size, 
                                                num_workers=num_workers, shuffle=True)
    
    # The Training set is the combination of the two remaining folds
    
    else:
        # Specify where the folders containing the training signals are for the dataset
        data1 = torchvision.datasets.DatasetFolder(data_dir[0], loader = torch.load, extensions = ".pt")
        data2 = torchvision.datasets.DatasetFolder(data_dir[1], loader = torch.load, extensions = ".pt")
        
        # Concatenate the datasets
        train_data = torch.utils.data.ConcatDataset((data1,data2))

        # Prepare data loaders
        # Signals are loaded in batch sizes as specified 
        loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                                num_workers=num_workers, shuffle=True)

    return loader

In [4]:
def three_fold_cross_sets(base_dir,fold1_dir,fold2_dir,fold3_dir,test_dir, batch_size =  64, num_workers = 0):
    '''
    # Loads the datasets in a three cross fold validation method
    base_dir - directory containing the rats data and signal class
    fold1_dir,fold2_dir,fold3_dir,test_dir - directories of the fold 1,2,3 and testing fold
    batch_size - how many signals are loaded per batch in the data loaders
    num_workers - number of processes loading batches in parallel
    
    Types
    base_dir,fold1_dir,fold2_dir,fold3_dir,test_dir - string
    batch_size - int
    num_workers - int
    '''
    #Fold 1
    train_set_1 = prepare_dataset([fold1_dir,fold2_dir], batch_size, num_workers)
    valid_set_1 = prepare_dataset(fold3_dir, batch_size, num_workers, "Val")

    #Fold 2

    train_set_2 = prepare_dataset([fold1_dir,fold3_dir], batch_size, num_workers)
    valid_set_2 = prepare_dataset(fold2_dir, batch_size, num_workers, "Val")

    #Fold 3
    train_set_3 = prepare_dataset([fold2_dir,fold3_dir],batch_size, num_workers)
    valid_set_3 = prepare_dataset(fold1_dir, batch_size, num_workers, "Val")

    test_set = prepare_dataset(test_dir, batch_size, num_workers, "Test")
    
    return train_set_1, valid_set_1, train_set_2, valid_set_2, train_set_3, valid_set_3, test_set

In [5]:
class EarlyStopping():
    '''
    Class for early stopping criteria, which will stop training if 
    validation loss stops increasing is is increasing very slowly
    
    min_delta - The minimum change in loss that can increase tolerance
    Tolerance - Number of epochs that decrease or (small increase) 
                in loss occurs for. 
    '''
    def __init__(self, tolerance=5, min_delta=0):

        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False

    def __call__(self, prev_val_loss, curr_val_loss):
        
        # Check if loss increases more than min_delta
        if (curr_val_loss - prev_val_loss) >= self.min_delta:
            self.counter +=1 # If loss increases, increase counter by one
            
            if self.counter >= self.tolerance:  
                self.early_stop = True # If counter equals or exceeds tolerance stop training
        
        # Rest counter to 0  if loss decreases less than min_delta
        else:
            self.counter = 0

In [6]:
def train_network(net, Trainset, Validset, num_classes, classes, learning_rate=0.01, num_epochs=30, early_stop =5, use_cuda = True, min_delta = 0, min_epochs = 30, title = None):
    '''
    net - network to be trained
    Trainset - Training data
    Validation - Data
    num_classes - number of potential output classes
    classes - possible class outputs
    learning_rate - learning rate for optimizer
    num_epochs - number of epochs to train for 
    early_stop - number of epochs for early stopping criterion
    use_cuda - leverage gpu for training
    min_delta - amount of change in validation loss required to trigger early stop
    title - title of network
    
    Types
    net - Pytorch Neural Network Object
    Trainset, Validset - Pytorch Dataloader Object
    num_classes - int
    classes - list of strings
    learning_rate - float
    num_epochs - int
    early_stop - int
    use_cude - bool
    min_delta - float
    title - string or None
    '''
    ########################################################################
    # Fixed PyTorch random seed for reproducible result
    torch.manual_seed(1000)
    ########################################################################
    # Define the Loss function and optimizer
    # The loss function will be Binary Cross Entropy (BCE). In this case we
    # will use the BCEWithLogitsLoss which takes unnormalized output from
    # the neural network and scalar label.
    # Optimizer will be SGD with Momentum.
    criterion = nn.CrossEntropyLoss() # Using Cross Entropy Loss
#     criterion = nn.NLLLoss() # Using Cross Entropy Loss

    optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.8)
    ########################################################################
    # Move model to gpu if available
    device = torch.device("cuda" if use_cuda and torch.cuda.is_available() else "cpu")
    net.to(device)
    ########################################################################
    # Set up some numpy arrays to store the training/test loss/accuracy
    train_acc = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_acc = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    ########################################################################
    # Train the network
    # Loop over the data iterator and sample a new batch of training data
    # Get the output from the network, and optimize our loss function.
    start_time = time.time()
    early_stopping = EarlyStopping(tolerance=early_stop, min_delta = min_delta)

    for epoch in tqdm(range(num_epochs)):  # loop over the dataset multiple times
        total_train_loss = 0.0
        total_train_acc = 0.0
        total_epoch = 0
        print("EPOCH: " + (str(epoch + 1)))
        for i, data in tqdm(enumerate(Trainset, 0)):
            
            # Turn on training mode, *turns on batch normalization and dropout
            net.train() #*****************************#

            # Get the inputs
            inputs, labels = data

        # Check if gpu is available and flagged for use
            if use_cuda and torch.cuda.is_available():
                inputs = inputs.double()
                inputs = inputs.cuda() # copy image batch to gpu memory
                labels = labels.cuda() # copy label batch to gpu memory

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass, backward pass, and optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)

            ### uncomment the following lines and comment the above two lines if auxiliary outputs are present
#             outputs = net(inputs)
#             output1 = outputs[0]
#             output2 = outputs[1]
#             loss1 = criterion(output1, labels)
#             loss2 = criterion(output2, labels)

#             outputs = output1 
#             print(outputs)
#             loss = loss1 + loss2*0.3

            loss.backward()
            optimizer.step()

            # Calculate the statistics
            guess = torch.argmax(outputs, axis=1)
            corr = (guess).squeeze() == labels
            corr = sum(corr)
            total_train_acc += corr
            total_train_loss += loss.item()
            total_epoch += len(labels)        


        train_acc[epoch] = (float(total_train_acc)) / total_epoch
        train_loss[epoch] = float(total_train_loss) / (i+1)
        val_acc[epoch], val_loss[epoch] = evaluate(net, criterion, Validset, use_cuda = True)
        if epoch >= min_epochs and val_acc[epoch] > 0.85:
            early_stopping(val_loss[epoch-1], val_loss[epoch])
        
        
        if early_stopping.early_stop:
            break
            print('Finished Training')
        print(("Epoch {}: Train Acc: {}, Train loss: {} |"+
                "Validation Acc: {}, Validation loss: {}").format(
                epoch + 1,
                train_acc[epoch],
                train_loss[epoch],
                val_acc[epoch],
                val_loss[epoch]))

        # Save the current model (checkpoint) to a file every 5 epochs
        if val_acc[epoch] > 0.8:
            torch.save(net.state_dict(), "model_checkpoints/" + title + "_epochno_" + str(epoch))
            
            end_time = time.time()

    # SAVE FINAL MODEL
    end_time = time.time()
    
    train_acc = train_acc[:epoch]
    train_loss = train_loss[:epoch]
    val_acc = val_acc[:epoch]
    val_loss = val_loss[:epoch]
    
    if not os.path.isdir("model_checkpoints/" + rat + "//"):
        os.mkdir("model_checkpoints/" + rat + "//")
    torch.save(net.state_dict(), "model_checkpoints/" + title + "_final")
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
    # Write the train/val loss/acc into CSV file for plotting later
    epochs = np.arange(1, num_epochs + 1)
    np.savetxt("{}_train_acc.csv".format("model_checkpoints//" + title), train_acc)
    np.savetxt("{}_train_loss.csv".format("model_checkpoints//" + title), train_loss)
    np.savetxt("{}_val_acc.csv".format("model_checkpoints//"+ title), val_acc)
    np.savetxt("{}_val_loss.csv".format("model_checkpoints//" + title), val_loss)

In [7]:
def evaluate(net, criterion, loader, use_cuda = True):
    """ Evaluate the network on the validation set.

    Args:
     net: PyTorch neural network object
     criterion: The loss function
     loader: PyTorch data loader for the validation set
     use_cuda: Bool selecting whether to evaluate on the GPU or not
    Returns:
     acc: A scalar for the avg classification acc over the validation set
     loss: A scalar for the average loss function over the validation set
    """

    ########################################################################
    # Move model to gpu if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net.to(device)
    net = net.cuda()

    # Initialize acc, loss, and epoch_no
    total_acc = 0.0
    total_loss = 0.0
    total_epoch = 0.0

    for i, data in enumerate(loader, 0):
        # Turn on Evaluation mode, *turns off batch normalization and dropout
        net = net.eval() #*****************************#

        # Takes data from train loader and splits into labels and inputs
        inputs, labels = data

        # Check if GPU is available and selected
        if use_cuda and torch.cuda.is_available():
            inputs = inputs.cuda() # Returns of copy of the image batch to the GPU memory
            labels = labels.cuda() # Returns of copy of the labels  to the GPU memory

        # Forward pass

        outputs = net(inputs.double())
#         outputs = outputs[0] # uncomment this if auxiliary outputs are present
        loss = criterion(outputs, labels)
    #         outputs = outputs[0] # uncomment this if auxiliary outputs are present

        # Calculate rate of incorrect predictions
        guess = torch.argmax(outputs, axis=1)
        corr = (guess).squeeze() == labels
        corr = sum(corr)

        # Update acc, loss, epoch number
        total_acc += corr
        total_loss += loss.item()
        total_epoch += len(labels)

    # Compute acc and loss
    acc = float(total_acc) / total_epoch
    loss = float(total_loss) / (i + 1)
    return acc, loss

In [8]:
def cf_mat_gen(net, test_set):
    '''
    Generates Confusion Matrix
    
    net - neural network
    test_set - test set
    
    Types
    net - Pytorch Neural Network Object
    test_set - pytorch dataloader object
    '''
    guesses = []
    labels = []
    net.eval()
    net = net.cpu()
    
    # Evaluate model for generating confusion matrix
    for i in test_set: # Iterate through batches
        batch = (i[0])
        num,__,_ = batch.shape
        label_batch = (i[1])

        # Evaluate
        outputs = (net(batch)).detach().numpy()
        guess = np.argmax(outputs, axis=1)
        guesses += list(guess)
        labels += list(np.array(label_batch))

    # Generate Confusion Matrix
    cf_mat = confusion_matrix(guesses, labels)
    return cf_mat

In [10]:
def accuracy(array):
    '''
    Accuracy Metric Function
    '''
    total_samples = np.sum(array)/1.
    correct = np.sum(np.multiply(array,np.eye(3)))

    accuracy = round(correct/total_samples*100,2)
    return(accuracy)

def recall(array, index):
    '''
    Recall Metric Function
    '''
    num = array[index,index]
    den = np.sum(array[index,:])
    
    return(num/den)

def precision(array, index):
    '''
    Precision Metric Function
    '''
    num = array[index,index]
    den = np.sum(array[:,index])
    
    return(num/den)

def macro_f1(array):
    '''
    Macro F1 Metric Function
    '''
    recall_arr = []
    precision_arr = []

    # Compute Precision and Recall for each class
    for i in range(3):
        recall_arr.append(recall(array, i))
        precision_arr.append(precision(array, i))

    # Get an F1 Score for each class
    per_class_f1 = []
    for i in range(3):
        per_class_f1.append((2*recall_arr[i]*precision_arr[i])/(recall_arr[i] + precision_arr[i]))

    # Macro F1    
    f1 = sum(per_class_f1)/3
    return f1

In [11]:
# Paste the Model you want to run here
class MSCB(nn.Module):
    def __init__(self, small_kernel, medium_kernel, large_kernel, num_filters):
        super(MSCB, self).__init__()
        self.name = "MSCB"

        # Define Small Path
        self.convS = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = small_kernel, padding = 'same')
        self.MPoolS = nn.MaxPool1d(kernel_size = small_kernel, stride = 5, padding = int(small_kernel/2 - 1))
        
        # Define Medium Path
        self.convM = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = medium_kernel, padding = 'same')
        self.MPoolM = nn.MaxPool1d(kernel_size = medium_kernel, stride = 5, padding = int(medium_kernel/2 - 1))
        
        # Define Large Path
        self.convL = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = large_kernel, padding = 'same')
        self.MPoolL = nn.MaxPool1d(kernel_size = large_kernel, stride = 5, padding = int(large_kernel/2 - 1))
        
        # MPool first
        self.MPool = nn.MaxPool1d(kernel_size = 3, stride = 5)
        self.conv = nn.Conv1d(in_channels = 56, out_channels = 128, kernel_size = 24, padding = 'same')
        
        # Post Concatenation
        self.conv2 = nn.Conv1d(in_channels = 3*num_filters + 128, out_channels = 64, kernel_size = 112, padding = 'same')
        self.MPool2 = nn.MaxPool1d(kernel_size = 3, stride = 5)
        self.fc1 = nn.Linear(in_features = 256, out_features = 400)
        self.Dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features = 400, out_features = 1024)
        self.fc3 = nn.Linear(in_features = 1024, out_features = 3)

    def forward(self, x):
        ### Feature Learning Head
        
        # Reshape Tensor
        x = torch.moveaxis(x,2,1)
        x = torch.tensor(x, dtype=torch.float32)

        # Parrallel Convolution Pathways
        x_S = self.MPoolS(F.relu(self.convS(x)))
        x_M = self.MPoolM(F.relu(self.convM(x)))
        x_L = self.MPoolL(F.relu(self.convL(x)))
        x_O = F.relu(self.conv(self.MPool(x)))
        
        # Post Concatenation
        x = torch.cat((x_S,x_M,x_L,x_O),1)
        x = self.MPool2(F.relu(self.conv2(x)))
        
        # Flattening
        x = x.view(-1,256)

        #Classification Head
        x = F.relu(self.fc1((x)))
        x = self.Dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [12]:
num_filters = 64
small_kernel = 1
medium_kernel = 3
large_kernel = 5


learning_rate = 0.001
num_epochs = 80
use_cuda = True
early_stop = 0
min_delta = -0.025
min_epochs = 30
num_workers = 8
batch_size = 64

In [13]:
count = 1
# for kernels in kernel_set: # Iterate through each potential kernel set
#     small_kernel, medium_kernel, large_kernel = kernels
#     set_no = "\nKernel set " + str(count) + "\n"

for i in [2,3,4,5,6,7,8,9,10]: # Iterate through each rat      

    rat = "Rat " + str(i)
    if not os.path.isdir("model_checkpoints//" + rat + "//"): # Make folder to save results
        os.mkdir("model_checkpoints//" + rat + "//")

    # Generate a base title for the model
    base_title = rat + "//" + rat + ("kernels_1_3_5")

    print(rat)

    # Specify directory of data
    base_dir = "M:\Peripheral Nerve Studies\MCC Projects\Aseem G\Models\Pytorch\Data\Spike 100\Minimum Dataset\\" + rat + "\\"
#     base_dir = "D:\Aseem\Spike Shape 100\\" + rat + "\\"

    # Specify directory of each fold
    fold1_dir = base_dir + "Fold1"
    fold2_dir = base_dir + "Fold2"
    fold3_dir = base_dir + "Fold3"
    test_dir = base_dir + "Test"

    # Load datasets
    train_set_1, valid_set_1, train_set_2, valid_set_2, train_set_3, valid_set_3, test_set = three_fold_cross_sets(base_dir,fold1_dir,fold2_dir,fold3_dir,test_dir, batch_size, num_workers)

    # Train Fold 1
    fold = rat + "--" + "Fold1"
    print("Starting Fold 1")
    title = base_title + "Fold_1"
    model = MSCB(small_kernel, medium_kernel, large_kernel, num_filters)
    train_network(model, train_set_1, valid_set_1, 3, ["DF", "PF", "Prick"], learning_rate, num_epochs, early_stop, use_cuda, min_delta, min_epochs, title)

    # Generate confusion matrices
#     cf_mat_train1 = cf_mat_gen(model, train_set_1)
#     cf_mat_val1 = cf_mat_gen(model, valid_set_1)

    torch.cuda.empty_cache() # Empty GPU Cache

    # Train Fold 2
    fold = "Fold2"
    print("Starting Fold 2")
    title = base_title + "Fold_2"
    model = MSCB(small_kernel, medium_kernel, large_kernel, num_filters)
    train_network(model, train_set_2, valid_set_2, 3, ["DF", "PF", "Prick"], learning_rate, num_epochs, early_stop, use_cuda, min_delta, min_epochs, title)

    # Generate confusion matrices
#     cf_mat_train2 = cf_mat_gen(model, train_set_2)
#     cf_mat_val2 = cf_mat_gen(model, valid_set_2)

    torch.cuda.empty_cache() # Empty GPU Cache

    # Train Fold 3
    fold = "Fold3"
    print("Starting Fold 3")
    title = base_title + "Fold_3"
    model = MSCB(small_kernel, medium_kernel, large_kernel, num_filters)
    train_network(model, train_set_3, valid_set_3, 3, ["DF", "PF", "Prick"], learning_rate, num_epochs, early_stop, use_cuda, min_delta, min_epochs, title)

    # Generate confusion matrices
#     cf_mat_train3 = cf_mat_gen(model, train_set_3)
#     cf_mat_val3 = cf_mat_gen(model, valid_set_3)


    # Write Training Summary File
    f = open("model_checkpoints//" + rat + "//" + rat + ("kernels_1_3_5") + "_summary.txt", "a")
    f.write(rat + " MSCB - DATA_AUG_REPEAT - kernels_1_3_5")
    f.write(("\nSmall Kernel = {}\nMedium Kernel = {}\nLarge Kernel = {}\nNum Filters = {}\nLearning Rate = {}\nPatience = {}\nBatch Size = {}\nMin Delta = {}\nMin Epochs = {}\n").format(small_kernel,
                                                                                                                               medium_kernel,
                                                                                                                               large_kernel,
                                                                                                                               num_filters,
                                                                                                                               learning_rate,
                                                                                                                               early_stop,
                                                                                                                               batch_size,
                                                                                                                               min_delta,
                                                                                                                               min_epochs))
#     f.write("\nFold_1\n")
#     f.write("Training\n")
#     f.write(str(cf_mat_train1))
#     f.write("\nTraining Accuracy: " + str(accuracy(cf_mat_train1)) + "%\n")
#     f.write("Training Macro F1 Score: " + str(macro_f1(cf_mat_train1)))
#     f.write("\nValidation\n")
#     f.write(str(cf_mat_val1))
#     f.write("\nValidation Accuracy: " + str(accuracy(cf_mat_val1)) + "%\n")
#     f.write("Validation Macro F1 Score: " + str(macro_f1(cf_mat_val1)))


#     f.write("\n\nFold_2\n")
#     f.write("Training\n")
#     f.write(str(cf_mat_train2))
#     f.write("\nTraining Accuracy: " + str(accuracy(cf_mat_train2)) + "%\n")
#     f.write("Training Macro F1 Score: " + str(macro_f1(cf_mat_train2)))
#     f.write("\nValidation\n")
#     f.write(str(cf_mat_val2))
#     f.write("\nValidation Accuracy: " + str(accuracy(cf_mat_val2)) + "%\n")
#     f.write("Validation Macro F1 Score: " + str(macro_f1(cf_mat_val2)))

#     f.write("\n\nFold_3\n")
#     f.write("Training\n")
#     f.write(str(cf_mat_train3))
#     f.write("\nnTraining Accuracy: " + str(accuracy(cf_mat_train3)) + "%\n")
#     f.write("Training Macro F1 Score: " + str(macro_f1(cf_mat_train3)))
#     f.write("\nValidation\n")
#     f.write(str(cf_mat_val3))
#     f.write("\nValidation Accuracy: " + str(accuracy(cf_mat_val3)) + "%\n")
#     f.write("Validation Macro F1 Score: " + str(macro_f1(cf_mat_val3)))
    f.close()
    torch.cuda.empty_cache() # Empty GPU Cache

count += 1

Rat 2
Starting Fold 1


  0%|          | 0/80 [00:00<?, ?it/s]

EPOCH: 1


0it [00:00, ?it/s]

C:\Users\GillA\AppData\Local\Temp\ipykernel_19444\2294328818.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)
C:\Users\GillA\.conda\envs\Aseem\lib\site-packages\torch\nn\modules\conv.py:303: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Convolution.cpp:883.)
  return F.conv1d(input, weight, bias, self.stride,


...

Epoch 78: Train Acc: 0.5934819897084048, Train loss: 1.0293144963004373 |Validation Acc: 0.41052029731275014, Validation loss: 1.0763266384601593
EPOCH: 79


0it [00:00, ?it/s]

Epoch 79: Train Acc: 0.5860491709548313, Train loss: 1.0235595909031954 |Validation Acc: 0.4528301886792453, Validation loss: 1.0678951229367937
EPOCH: 80


0it [00:00, ?it/s]

Epoch 80: Train Acc: 0.5934819897084048, Train loss: 1.0160711353475398 |Validation Acc: 0.4562607204116638, Validation loss: 1.0652969862733568
Total time elapsed: 2369.28 seconds
